In [1]:
import serial
import datetime

In [2]:
serialPort = serial.Serial(port = "COM6", baudrate = 3000000, bytesize = 8, timeout = 0, stopbits = serial.STOPBITS_ONE)
t_before = datetime.datetime.now()

In [3]:
i = 0
repeat_max = 10000
buffSizes = []
stringTotal = bytearray() #""
serialString = bytearray() #""
#buff_size = serialPort.in_waiting
while (serialPort.in_waiting >= 0) and (i < repeat_max):
    serialString = serialPort.read(serialPort.in_waiting)
    stringTotal.extend(serialString) #stringTotal += str(serialString)
    if serialPort.in_waiting > 0:
        i += 1
#    buffSizes.append(buff_size)
#    buff_size = serialPort.in_waiting
    
t_after = datetime.datetime.now()
t_diff = (t_after - t_before).total_seconds()

In [4]:
#stringTotal = stringTotal.replace("\'", "")
#stringTotal = stringTotal.replace("b", "")
print("time elapsed:", t_diff, ", bytes: ", len(stringTotal), ", packets/sec:", len(stringTotal)/t_diff, ", Incoming: ...", stringTotal[-21:])

time elapsed: 10.347325 , bytes:  3101167 , packets/sec: 299707.12237220735 , Incoming: ... bytearray(b'^_`abcdefghijklmnopqr')


In [5]:
numCount = {}
for i in stringTotal:
    if i in numCount.keys():
        numCount[i] += 1
    else:
        numCount[i] = 1
print(numCount)

{37: 12115, 38: 12115, 39: 12115, 40: 12115, 41: 12115, 42: 12115, 43: 12115, 44: 12115, 45: 12115, 46: 12115, 47: 12115, 48: 12115, 49: 12115, 50: 12115, 51: 12115, 52: 12115, 53: 12115, 54: 12115, 55: 12115, 56: 12115, 57: 12115, 58: 12115, 59: 12115, 60: 12115, 61: 12115, 62: 12115, 63: 12115, 64: 12115, 65: 12115, 66: 12115, 67: 12115, 68: 12115, 69: 12115, 70: 12115, 71: 12115, 72: 12115, 73: 12115, 74: 12115, 75: 12115, 76: 12115, 77: 12115, 78: 12115, 79: 12115, 80: 12115, 81: 12115, 82: 12115, 83: 12115, 84: 12115, 85: 12115, 86: 12115, 87: 12115, 88: 12115, 89: 12115, 90: 12115, 91: 12115, 92: 12115, 93: 12115, 94: 12115, 95: 12115, 96: 12115, 97: 12115, 98: 12115, 99: 12115, 100: 12115, 101: 12115, 102: 12115, 103: 12115, 104: 12115, 105: 12115, 106: 12115, 107: 12115, 108: 12115, 109: 12115, 110: 12115, 111: 12115, 112: 12115, 113: 12115, 114: 12115, 115: 12114, 116: 12114, 117: 12114, 118: 12114, 119: 12114, 120: 12114, 121: 12114, 122: 12114, 123: 12114, 124: 12114, 125: 1

In [6]:
list(stringTotal)
diff = [t - s for s, t in zip(stringTotal, stringTotal[1:])]
skipped = [i for i in diff if i not in [1, -0xff]]
skippedIndices = [i for i, x in enumerate(diff) if x not in [1, -0xff]]

In [7]:
print(skippedIndices)
#print(diff)
#filledBuff = [i for i in buffSizes if i != 0]
#print(filledBuff)
#print(len(buffSizes), buffSizes.count(0))

[1688452]


In [26]:
for i in skippedIndices:
    hexSkipped = stringTotal[i - 4 : i + 6].hex()
    skipRange = [hexskipped[i:i+2] for i in range(0,len(hexSkipped), 2)]
    a = int(skipRange[5], 16)
    b = int(skipRange[4], 16)
    print(skipRange, (a - b) & 0xff)

['a5', 'a6', 'a7', 'a8', 'a9', '09', '0a', '0b', '0c', '0d'] 96


In [24]:
0xf9 - 0x99

96

class ReadLine:
    def __init__(self, s, l):
        self.buf = bytearray()
        self.s = s
#        self.lim = l
    
    def readline(self):
        i = self.buf.find(b"\n")
        if i >= 0:
            r = self.buf[:i+1]
            self.buf = self.buf[i+1:]
            return r
        for k in range(5000):#while True:
            i = max(1, min(2048, self.s.in_waiting))
            data = self.s.read(i)
            i = data.find(b"\n")
            if i >= 0:
                r = self.buf + data[:i+1]
                self.buf[0:] = data[i+1:]
                return r
            else:
                self.buf.extend(data)

#serialPort = serial.Serial(port = "COM6", baudrate = 3000000, bytesize = 8, timeout = 0, stopbits = serial.STOPBITS_ONE)
rl = ReadLine(serialPort, 500000)

while True:
    print(rl.readline())